请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

# ChatGLM-6B简介

ChatGLM-6B 是一个开源的、支持中英双语的对话语言模型，基于 General Language Model (GLM) 架构，具有 62 亿参数。结合模型量化技术，用户可以在消费级的显卡上进行本地部署（INT4 量化级别下最低只需 6GB 显存）。 ChatGLM-6B 使用了和 ChatGPT 相似的技术，针对中文问答和对话进行了优化。经过约 1T 标识符的中英双语训练，辅以监督微调、反馈自助、人类反馈强化学习等技术的加持，62 亿参数的 ChatGLM-6B 已经能生成相当符合人类偏好的回答。

具体参考ChatGLM-6B的GitHub
https://github.com/THUDM/ChatGLM-6B


# 1.项目介绍：

为了方便大家学习、使用ChatGLM-6B。针对不同场景开发、测试ChatGLM 的功能。开发了多个函数:


## 问答
ask_glm(next_inputs,input_length,output_length)，这个函数可以一次性调用ChatGLM，虽然功能简单但是可以方便的与其他功能集成。适合大家利用开发自己的功能。

运行效果如图：
![](https://ai-studio-static-online.cdn.bcebos.com/b8da9e50b5e447d29cbf2d08f2c3fd1c4757c5b8862347a6bb585c244d2b752a)


## 多次交互
chat_with_glm(input_prompt,input_length=2048,output_length=256)，则个函数可以与ChatGLM进行交谈，是命令行界面。
运行效果如图：
![](https://ai-studio-static-online.cdn.bcebos.com/4adf7a66145d40f3bac8fcff4c9dca3e36a6a32a4bb44766bd609f4b75ab3c12)


## 图形界面，问答
ask_glm_widgets(model,tokenizer,text='你好')，这个函数是使用ipywidgets界面与ChatGLM一次性交互。
运行效果如图：
![](https://ai-studio-static-online.cdn.bcebos.com/7049dcdfc6524891979c9c51e17c58f6a774aa4b24b5401b955cebe22c0a8c95)


## 图形界面，交互
chat_glm_widgets(model,tokenizer)，这个函数是使用ipywidgets界面与ChatGLM交谈。
运行效果如图：
![](https://ai-studio-static-online.cdn.bcebos.com/0c816f57f96c43e0b752efe0e8b6a5e280165b64fb094e118cbe5cc0ab1b738a)



分别用于一次性交流和多次交谈以及带图形界面的交流。

注：

编写调用GLM程序的时候借鉴了宇宙物语的 “基于ChatGLM-6B模型 + prompt实现角色扮演功能” 项目内容。 

以及PaddleNLP的例子代码：
https://github.com/PaddlePaddle/PaddleNLP/tree/develop/examples/language_model/glm


# 2.模型安装 （安装后重启内核）
安装后点击重启内核。

这部分内容只需要第一次运行的时候执行一遍，后续每次运行就不需要再执行了。

In [ ]:
import paddle
print(paddle.version.cuda())
!unzip paddlenlp.zip
!cp -Rf paddlenlp /home/aistudio/.data/webide/pip/lib/python3.7/site-packages/paddlenlp
!python -m pip install paddlepaddle-gpu==0.0.0.post112 -f https://www.paddlepaddle.org.cn/whl/linux/gpu/develop.html --user
#  注意安装后重启内核

# 3.模型调用

封装了4个函数:

1，ask_glm(next_inputs,input_length,output_length)

直接调用ChatGLM模型，输入参数：初始prompt, 最长输入长度，最长输出长度



2，chat_with_glm(input_prompt,input_length=2048,output_length=256)

与ChatGLM多次交互 输入参数：初始prompt, 最长输入长度，最长输出长度。交谈时，输入每一句后点击回车即可。需要退出时，输入：'1','退出','exit','bye','结束'都可以


3，ask_glm_widgets(model,tokenizer,text='你好')

ipywidgets图形界面交互，与ChatGLM交流，不展示历史消息

4，chat_glm_widgets(model,tokenizer)

ipywidgets图形界面 与ChatGLM交流，发送及展示历史消息



分别用于一次性交流和多次交谈。

In [1]:
import warnings 
import paddle
import os
from IPython.display import clear_output as clear
warnings.filterwarnings("ignore")

def get_model(local_model_path):
    from paddlenlp.transformers import (
        ChatGLMConfig,
        ChatGLMForConditionalGeneration,
        ChatGLMTokenizer,
    )
    #读取原始的chatglm-6b模型
    model_name_or_path = local_model_path
    
    tokenizer = ChatGLMTokenizer.from_pretrained(model_name_or_path)
    config = ChatGLMConfig.from_pretrained(model_name_or_path)
    paddle.set_default_dtype(config.paddle_dtype)

    model = ChatGLMForConditionalGeneration.from_pretrained(
        model_name_or_path,
        tensor_parallel_degree=paddle.distributed.get_world_size(),
        tensor_parallel_rank=0,
        load_state_as_np=True,
        dtype=config.paddle_dtype,
    )

    model.eval()
    return model,tokenizer

def convert_example(example, tokenizer, is_test=True):
    query = example["user_input"]
    response = example["sys_output"]
    history = example.get("history", None)
    if history is None or len(history) == 0:
        prompt = query
    else:
        prompt = ""
        for i, (old_query, old_response) in enumerate(history):
            prompt += "[Round {}]\n问：{}\n答：{}\n".format(i, old_query, old_response)
        prompt += "[Round {}]\n问：{}\n答：".format(len(history), query)
    return prompt

# 问glm问题
# 输入参数：初始prompt, 最长输入长度，最长输出长度
def ask_glm(model,tokenizer,next_inputs,input_length,output_length):
    inputs = tokenizer(
        next_inputs,
        return_tensors="np",
        padding=True,
        max_length=input_length,
        truncation=True,
        truncation_side="left",
    )
    input_map = {}
    for key in inputs:
        input_map[key] = paddle.to_tensor(inputs[key])

    infer_result = model.generate(
        **input_map,
        decode_strategy="sampling",
        top_k=1,
        # top_p =5,
        max_length=output_length,
        use_cache=True,
        use_fast=True,
        use_fp16_decoding=True,
        repetition_penalty=1,
        temperature = 0.95,
        length_penalty=1,
    )[0]
    output = ''
    result = []
    for x in infer_result.tolist():
        res = tokenizer.decode(x, skip_special_tokens=True)
        res = res.strip("\n")
        result.append(res)
        print(res)
        output = output + res
    return output

# 与GLM交谈
# 输入参数：初始prompt, 最长输入长度，最长输出长度
# 交谈时，输入每一句后点击回车即可
# 需要退出时，输入：'1','退出','exit','bye','结束'都可以
def chat_with_glm(model,tokenizer,input_prompt,input_length=2048,output_length=256):
    start = 1

    while(1):
        if(start):
            prompt = input_prompt
            example = {'user_input':'' , 'sys_output':'' ,'history':[]}
            user_input = prompt
            print('问：'+prompt)
            print('答：',end='')
            start = 0
        else:
            user_input = input()
            if(user_input in ['1','退出','exit','bye','结束','再见']):
                print('退出')
                break
            os.system('cls' if os.name == 'nt' else 'clear')
            clear()
            print(user_input)
            example['user_input'] = user_input
            prompt = convert_example(example, tokenizer, is_test=True)
            if(len(prompt)>input_length):
                prompt = prompt[-input_length:-1]
            print(prompt,end='')
        inputs = tokenizer(
            prompt,
            return_tensors="np",
            padding=True,
            max_length=input_length,
            truncation=True,
            truncation_side="left",
        )
        input_map = {}
        for key in inputs:
            input_map[key] = paddle.to_tensor(inputs[key])

        infer_result = model.generate(
            **input_map,
            decode_strategy="sampling",
            top_k=1,
            max_length=output_length,
            use_cache=True,
            use_fast=True,
            use_fp16_decoding=True,
            repetition_penalty=1,
            temperature = 0.95,
            length_penalty=1,
        )[0]
        res = tokenizer.decode(infer_result.tolist()[0], skip_special_tokens=True)
        res = res.strip("\n")
        example['sys_output'] = res
        example['history'].append((user_input,res))
        print(res)

import ipywidgets as widgets
# 图形界面 与ChatGLM交流，不展示历史消息
def ask_glm_widgets(model,tokenizer,text='你好'):
    # 创建两个Textarea小部件
    txtprompt = widgets.Textarea(description='提示:',rows=15,layout=widgets.Layout(width='48%'))
    txtresult = widgets.Textarea(description='结果:',rows=15,layout=widgets.Layout(width='48%'))

    txtprompt.value=text

    # 创建一个按钮小部件
    btngo = widgets.Button(description='生成',layout=widgets.Layout(width='50px'), button_style='info')

    # 定义按钮的点击事件处理程序
    def send_prompt(btn):
        result= ask_glm(model,tokenizer,txtprompt.value,2048,2048)
        txtresult.value = result

    # 绑定按钮的点击事件处理程序
    btngo.on_click(send_prompt)

    # 创建一个布局盒子，使用Flexbox布局
    box_layout = widgets.Layout(display='flex', flex_flow='row', align_items='center')

    # 将txtprompt、btngo和txtresult放入布局盒子中
    hbox = widgets.HBox([txtprompt, btngo, txtresult], layout=box_layout)

    # 显示布局盒子
    display(hbox)

# 图形界面 与ChatGLM交流，发送及展示历史消息
def chat_glm_widgets(model,tokenizer):
    # 创建一个Textarea小部件用于输入聊天消息
    # txtinput = widgets.Text(description='输入:', layout=widgets.Layout(width='95%'))
    txtinput = widgets.Textarea(description='输入:', layout=widgets.Layout(width='95%', height='150px'))

    # 创建一个Textarea小部件用于显示聊天历史
    txthistory = widgets.Textarea(description='历史:', layout=widgets.Layout(width='95%', height='200px'), disabled=True)

    # 创建一个按钮小部件用于发送消息
    btnsend = widgets.Button(description='发送', button_style='info')

    # 创建一个按钮小部件用于重置历史
    btnreset = widgets.Button(description='重置', button_style='info')

    # 定义发送消息的函数
    def send_message(_):
        # 获取用户输入的消息
        message = txtinput.value.strip()

        if message:
            # 添加用户输入的消息到聊天历史
            txthistory.value += f'问: {message}\n'

            # 调用本地部署的LLM并获取回答结果
            #result = ask_glm(message)
            #result = ask_glm(model,tokenizer,message,2048,2048)
            result = ask_glm(model,tokenizer,txthistory.value,2048,2048)

            # 添加LLM的回答到聊天历史
            txthistory.value += f'答: {result}\n\n'

            # 清空输入框
            txtinput.value = ''

    # 绑定发送消息按钮的点击事件处理程序
    btnsend.on_click(send_message)

    # 定义重置历史的函数
    def reset_history(_):
        # 清空聊天历史和输入框
        txthistory.value = ''
        txtinput.value = ''

    # 绑定重置历史按钮的点击事件处理程序
    btnreset.on_click(reset_history)

    # 将发送消息按钮和重置历史按钮放入水平布局盒子
    buttons_box = widgets.HBox([btnsend, btnreset], layout=widgets.Layout(justify_content='center'))

    # 创建一个垂直布局盒子，将小部件放入其中
    vbox = widgets.VBox([txtinput, buttons_box, txthistory])


    # 显示布局盒子
    display(vbox)

# 4.模型效果测试

针对不同场景进行测试，整体感觉效果是不错的。



In [ ]:
# 读取模型
model,tokenizer = get_model('data/data217141')

In [19]:
result=ask_glm(model,tokenizer,'中国的首都是哪个城市？',2048,2048)

中国的首都是北京。


In [4]:
result=ask_glm(model,tokenizer,'介绍一下Python语言',2048,2048)

Python是一种高级编程语言,具有简单易学、可读性强、功能强大、可移植性好等特点。Python最初由Guido van Rossum在1989年开发,并于1991年首次发布。

Python语言具有以下特点:

1. 简单易学:Python语法简洁明了,易于理解和学习,不需要掌握复杂的编程概念和技巧。

2. 可读性强:Python采用缩进作为代码标识符,使得代码可读性很强,同时也方便代码维护。

3. 功能强大:Python拥有大量的内置模块,可以用于各种编程任务,如数据处理、网络编程、Web开发、人工智能等。

4. 可移植性好:Python语言可以在不同的操作系统和硬件平台上运行,不需要进行特定的编译或改写。

5. 跨平台:Python可以在Windows、MacOS、Linux等多个操作系统上运行,同时也支持多种编程环境,如IDLE、PyCharm等。

Python语言广泛应用于各个领域,如科学计算、数据分析、人工智能、机器学习、自然语言处理、Web开发等。同时,Python语言也是开源社区的重要成员,有大量的开源库和框架可供使用。


In [6]:
chat_with_glm(model,tokenizer,'你是一个作者，我是一个编辑，我们一起设计一本图书',2048,2048)

我们设计一个面向儿童的关于友情的插画书，5页左右，请设计每页的内容。
[Round 0]
问：你是一个作者，我是一个编辑，我们一起设计一本图书
答：很高兴能和您一起设计一本图书!作为一个作者,我希望您能告诉我关于这本书的一些想法和计划,包括主题、情节、角色、风格和其他方面。作为一个编辑,我的职责是确保这本书的质量和准确性,并与您一起制定出版计划。我们可以一起讨论并制定一个详细的计划,以确保这本书能够达到预期的效果。
[Round 1]
问：我们来编辑一本儿童书吧
答：好的,那我们可以开始讨论了!首先,我们可以讨论一下您想要的主题、情节、角色和风格,并确定一些基本的想法。然后,我们可以进一步讨论细节,例如用词、句式、插图和排版等方面,以确保这本书的整体质量。最后,我们可以制定出版计划,并确定出版日期和预算。让我们一起开始吧!
[Round 2]
问：我们设计一个面向儿童的关于友情的插画书，5页左右，请设计每页的内容。
答：好的,让我们开始设计这本书吧!

第1页:这是一个关于友情的故事,告诉我们友谊的重要性。插图展示了两个好朋友在一起玩耍的场景,他们互相支持、帮助和尊重。文字告诉我们,友谊是一种无私的奉献,可以带来快乐和幸福。

第2页:在这个页中,我们继续介绍友谊的重要性,并讨论如何建立和维护好朋友关系。插图展示了两个好朋友在一起学习的场景,他们互相鼓励、帮助和支持,形成了良好的学习氛围。文字告诉我们,友谊是一种互相支持和帮助的关系,可以帮助人们克服困难,取得成功。

第3页:在这个页中,我们讨论友谊的另一种形式,即团队友谊。插图展示了两个好朋友在一起工作的场景,他们互相支持、帮助和尊重,形成了良好的团队氛围。文字告诉我们,友谊不仅是一种个人关系,也是一种团队关系,可以帮助人们更好地合作和完成任务。

第4页:在这个页中,我们讨论友谊的负面影响,即嫉妒和争吵。插图展示了两个好朋友之间发生争吵的场景,他们互相指责和批评,形成了不良的友谊关系。文字告诉我们,嫉妒和争吵是友谊中常见的问题,但如果不及时解决,会对友谊造成负面影响。

第5页:在这个页中,我们总结友谊的重要性,并鼓励孩子们珍惜友谊。插图展示了两个好朋友在一起庆祝的场景,他们互相拥抱和祝福,形成了良好的友谊关系。文字告诉我们,友谊是一种珍贵的财富,可以带来快乐和幸福,我们应该珍惜友谊,并与他人建立良好的关系。


 bye


退出


In [7]:
chat_with_glm(model,tokenizer,'你是一个数学老师，我是一个学生，我们一起在作算术题',2048,2048)

算错了
[Round 0]
问：你是一个数学老师，我是一个学生，我们一起在作算术题
答：好的,我很高兴能和您一起解决算术问题。请问您有什么问题或需要我的帮助吗?
[Round 1]
问：5x=12 x是多少？
答：将等式两边同时除以5,可以得到:

x = 12 / 5

将分数进行运算,得到:

x = 2.4

因此,x的值为2.4。
[Round 2]
问：5*8+9=？
答：5*8+9=43
[Round 3]
问：算错了
答：非常抱歉,我的回答有误。正确的答案应该是:

5*8+9=43

将等式两边同时乘以5,得到:

5*8+9=43

因此,5*8+9的值为43。


 bye


退出


In [3]:
ask_glm_widgets(model,tokenizer)

In [4]:
chat_glm_widgets(model,tokenizer)